In [2]:
#Import all libraries that will be used
import pandas as pd
import numpy as np

In [3]:
import warnings
from warnings import filterwarnings
filterwarnings('ignore')

In [4]:
#load the ratings and movies file into the dataframe
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')

In [5]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [7]:
movie_ratings = pd.merge(movies,ratings, on='movieId', how='inner')
movie_ratings.drop('timestamp', axis=1, inplace=True)
movie_ratings.head()

,movieId,title,genres,userId,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2,3.5
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3,4.0
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4,3.0
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8,4.0


In [8]:
#movie_rating_df = 
reviews = movie_ratings.groupby(['title'])['rating'].agg(['count','mean']).round(1)
reviews.head()

,count,mean
title,,
"""BLOW THE NIGHT!"" Let's Spend the Night Together (1983)",1,3.0
"""Great Performances"" Cats (1998)",179,2.9
#1 Cheerleader Camp (2010),9,2.1
#Captured (2017),2,3.8
#Female Pleasure (2018),3,3.7


In [9]:
movie_ratings.shape

(25000095, 5)

In [10]:
#movie_ratings.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25000095 entries, 0 to 25000094
Data columns (total 5 columns):
 #   Column   Dtype  
---  ------   -----  
 0   movieId  int64  
 1   title    object 
 2   genres   object 
 3   userId   int64  
 4   rating   float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.5 GB


In [11]:
#movie_ratings.describe().round()

,movieId,userId,rating
count,25000095.0,25000095.0,25000095.0
mean,21388.0,81189.0,4.0
std,39199.0,46792.0,1.0
min,1.0,1.0,0.0
25%,1196.0,40510.0,3.0
50%,2947.0,80914.0,4.0
75%,8623.0,121557.0,4.0
max,209171.0,162541.0,5.0


In [12]:
movie_ratings=movie_ratings.astype({'movieId':'int32','userId':'int32','genres':'category'})

In [13]:
#mov = pd.crosstab(index=movie_ratings['userId'], columns=movie_ratings['title'],values=movie_ratings['rating'], aggfunc='sum')

In [14]:
movie_ratings.shape

(25000095, 5)

In [15]:
ratings.shape

(25000095, 4)

In [16]:
ratings.nunique()

userId         162541
movieId         59047
rating             10
timestamp    20115267
dtype: int64

In [17]:
ratings.isna().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [18]:
#movie_ratings.nunique()

In [19]:
user_counts = movie_ratings['userId'].value_counts()
user_counts

72315     32202
80974      9178
137293     8913
33844      7919
20055      7488
          ...  
127695       20
152765       20
90815        20
70828        20
141949       20
Name: userId, Length: 162541, dtype: int64

In [20]:
valid_user_ids = user_counts[user_counts > 0].index
valid_user_ids

Int64Index([ 72315,  80974, 137293,  33844,  20055, 109731,  92046,  49403,
             30879, 115102,
            ...
            151863, 112112,  75766, 127773, 111735, 127695, 152765,  90815,
             70828, 141949],
           dtype='int64', length=162541)

In [21]:
filtered_ratings = movie_ratings[movie_ratings['userId'].isin(valid_user_ids)]

# Crosstab with the filtered DataFrame
#mov = pd.crosstab(index=filtered_ratings['userId'],
#                  columns=filtered_ratings['title'],
#                  values=filtered_ratings['rating'],
#                  aggfunc='sum')
#mov

In [22]:
batch_size = 5000  # Set the batch size
total_users = len(valid_user_ids)
num_batches = total_users // batch_size + 1
num_batches

33

In [23]:
#The error you're encountering, "MemoryError: Unable to allocate 469. KiB for an array with shape (6, 10000) and data type 
#float64," indicates that your code is running out of memory while trying to allocate memory for an array with shape 
#(6, 10000) and data type float64.

#The issue is most likely caused by the concatenation of DataFrames mov and batch_mov inside the loop. As the loop progresses, the mov DataFrame keeps growing as you concatenate more batches, eventually causing a memory error.

#To resolve this issue, you can modify your code to append each batch to a list and then concatenate them outside the loop. 
#Here's an updated version of your code that avoids the memory error:
#mov = None  # Initialize the result DataFrame

#for i in range(num_batches):
#    start_index = i * batch_size
#    end_index = start_index + batch_size
#    batch_users = valid_user_ids[start_index:end_index]

#    batch_mov = pd.crosstab(index=filtered_ratings[filtered_ratings['userId'].isin(batch_users)]['userId'],
#                            columns=filtered_ratings[filtered_ratings['userId'].isin(batch_users)]['title'],
#                            values=filtered_ratings[filtered_ratings['userId'].isin(batch_users)]['rating'],
#                            aggfunc='sum')

#    if mov is None:
#        mov = batch_mov
#    else:
#        mov = pd.concat([mov, batch_mov])

# The 'mov' DataFrame now contains the crosstab result in batches using pd.concat

In [ ]:
#user_rating_list = []  # Initialize the list to store batches
#user_rating = pd.DataFrame()

#for i in range(num_batches):
#    start_index = i * batch_size
#    end_index = start_index + batch_size
#    batch_users = valid_user_ids[start_index:end_index]
#    print("Running batch no: "+ str(i))

#    batch_mov = pd.crosstab(index=filtered_ratings[filtered_ratings['userId'].isin(batch_users)]['userId'],
#                            columns=filtered_ratings[filtered_ratings['userId'].isin(batch_users)]['title'],
#                            values=filtered_ratings[filtered_ratings['userId'].isin(batch_users)]['rating'],
#                            aggfunc='sum')

#    user_rating_list.append(batch_mov)  # Append each batch to the list

#user_rating = pd.concat(user_rating_list, ignore_index=True)  # Concatenate all batches


Running batch no: 0
Running batch no: 1
Running batch no: 2
Running batch no: 3
Running batch no: 4
Running batch no: 5
Running batch no: 6
Running batch no: 7
Running batch no: 8
Running batch no: 9
Running batch no: 10
Running batch no: 11
Running batch no: 12
Running batch no: 13
Running batch no: 14
Running batch no: 15
Running batch no: 16
Running batch no: 17
Running batch no: 18
Running batch no: 19
Running batch no: 20
Running batch no: 21
Running batch no: 22
Running batch no: 23
Running batch no: 24
Running batch no: 25
Running batch no: 26
Running batch no: 27
Running batch no: 28
Running batch no: 29
Running batch no: 30
Running batch no: 31
Running batch no: 32


In [24]:
output_dir = r"batch_files/"  # Specify the directory to store the sets

In [31]:

output_dir = r"batch_files/"  # Specify the directory to store the sets

for i in range(num_batches):
    start_index = i * batch_size
    end_index = start_index + batch_size
    batch_users = valid_user_ids[start_index:end_index]
    print("Running batch no: " + str(i))

    batch_mov = pd.crosstab(
        index=filtered_ratings[filtered_ratings['userId'].isin(batch_users)]['userId'],
        columns=filtered_ratings[filtered_ratings['userId'].isin(batch_users)]['title'],
        values=filtered_ratings[filtered_ratings['userId'].isin(batch_users)]['rating'],
        aggfunc='sum'
    )
    print("Writing batch no (" + str(i) + ") to disk")
    batch_mov.to_csv(output_dir + f"batch_{i}.csv", index=False)  # Write each batch to disk as a separate file


Running batch no: 0
Writing batch no (0) to disk
Running batch no: 1
Writing batch no (1) to disk
Running batch no: 2
Writing batch no (2) to disk
Running batch no: 3
Writing batch no (3) to disk
Running batch no: 4
Writing batch no (4) to disk
Running batch no: 5
Writing batch no (5) to disk
Running batch no: 6
Writing batch no (6) to disk
Running batch no: 7
Writing batch no (7) to disk
Running batch no: 8
Writing batch no (8) to disk
Running batch no: 9
Writing batch no (9) to disk
Running batch no: 10
Writing batch no (10) to disk
Running batch no: 11
Writing batch no (11) to disk
Running batch no: 12
Writing batch no (12) to disk
Running batch no: 13
Writing batch no (13) to disk
Running batch no: 14
Writing batch no (14) to disk
Running batch no: 15
Writing batch no (15) to disk
Running batch no: 16
Writing batch no (16) to disk
Running batch no: 17
Writing batch no (17) to disk
Running batch no: 18
Writing batch no (18) to disk
Running batch no: 19
Writing batch no (19) to disk


In [30]:
#reviews.to_csv(output_dir + f"batch_{0}.csv", index=False)  # Write each batch to disk as a separate file


In [22]:
num_batches

33

In [25]:

#print("Before concatenation")
user_rating_list = []  # Re-initialize the list to store DataFrames

for i in range(num_batches):
    print("Loading batch no (" + str(i) + ") from disk")
    batch_mov = pd.read_csv(output_dir + f"batch_{i}.csv")
    user_rating_list.append(batch_mov)  # Append each batch DataFrame to the list
    if i % 5==0:
        print("Appending batch no (" + str(i) + ") to user_rating")
        user_rating.append(user_rating_list)
        user_rating_list = []

#user_rating = pd.concat(user_rating_list, ignore_index=True)  # Concatenate all DataFrames from the list

Loading batch no (0) from disk


KeyboardInterrupt: 

In [1]:
user_rating.head()

NameError: name 'user_rating' is not defined